In [4]:
from metadata import *
G=graph
# same layout using matplotlib with no labels
plt.title('draw_networkx')
pos=nx.nx_agraph.graphviz_layout(G)
nx.draw(G, pos, with_labels=False, arrows=False)
plt.savefig('nx_test2.png')

/home/wangxinglu/anaconda2/lib/python2.7/site-packages/networkx/drawing/nx_pylab.py:126: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  b = plt.ishold()
/home/wangxinglu/anaconda2/lib/python2.7/site-packages/networkx/drawing/nx_pylab.py:138: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  plt.hold(b)
/home/wangxinglu/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:917: UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  warnings.warn(self.msg_depr_set % key)
/home/wangxinglu/anaconda2/lib/python2.7/site-packages/matplotlib/rcsetup.py:152: UserWarning: axes.hold is deprecated, will be removed in 3.0
  warnings.w

In [ ]:
nimgs={}
os.chdir('/mnt/nfs1703/kchen/imagenet-raw/')
for d in os.listdir('.'):
    nimgs[d]=len(os.listdir(d))

In [1]:
from utils import *
import utils
utils.init_dev(utils.get_dev())
utils.allow_growth()

%load_ext autoreload
# %reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from cifar100_train import *

Auto select gpu
 ID  GPU  MEM
--------------
  0  55%  66%
  1  62%  69%
  2  68%  69%
  3   0%  35%
  4   0%   0%
  5   0%   0%
  6   0%   1%
  7   0%   1%
available [4]

Using TensorFlow backend.



Auto select gpu
 ID  GPU  MEM
--------------
  0  41%  66%
  1  98%  69%
  2  99%  69%
  3   0%  35%
  4   0%   1%
  5   0%   0%
  6   0%   1%
  7   0%   1%
available [5]


In [2]:
from datasets.cifar100 import * 
import utils 
from datasets import cifar100
from cifar100_train import * 

batch_size=FLAGS.batch_size

# load the dataset
dataset = cifar100.get_split('train', FLAGS.data_dir)

# load batch of dataset
data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        num_readers=8,
        common_queue_capacity=40 * batch_size,
        common_queue_min=20 * batch_size)

images, labels = data_provider.get(['image', 'label'])


sess=tf.Session(config= utils.allow_growth_config()) 
_=tf.train.queue_runner.start_queue_runners(sess)


# lb=0
# while lb!=52:
im,lb=sess.run([images,labels])

from datasets.cifar100 import *
# a2b_map = {a:b for b,a in b2a_map.items()}
plt.figure(figsize=(2,2))
plt.imshow(im,)
plt.grid('off')
print lb
# blb=a2b_map[lb]
# print blb 
print fine_labels_human[lb]

In [2]:
c2f_map

{0: {4, 30, 55, 72, 95},
 1: {1, 32, 67, 73, 91},
 2: {54, 62, 70, 82, 92},
 3: {9, 10, 16, 28, 61},
 4: {0, 51, 53, 57, 83},
 5: {22, 39, 40, 86, 87},
 6: {5, 20, 25, 84, 94},
 7: {6, 7, 14, 18, 24},
 8: {3, 42, 43, 88, 97},
 9: {12, 17, 37, 68, 76},
 10: {23, 33, 49, 60, 71},
 11: {15, 19, 21, 31, 38},
 12: {34, 63, 64, 66, 75},
 13: {26, 45, 77, 79, 99},
 14: {2, 11, 35, 46, 98},
 15: {27, 29, 44, 78, 93},
 16: {36, 50, 65, 74, 80},
 17: {47, 52, 56, 59, 96},
 18: {8, 13, 48, 58, 90},
 19: {41, 69, 81, 85, 89}}

In [28]:
mapp={}
for c,fs in c2f_map.items():
    for f in fs:
        if coarse_labels_human[c] not in mapp: 
            mapp[coarse_labels_human[c]]={fine_labels_human[f]}
        else:
            mapp[coarse_labels_human[c]].add(fine_labels_human[f])
mapp            

In [3]:
FLAGS.batch_size=batch_size=1

In [24]:
tf.reset_default_graph()

Auto select gpu
 ID  GPU  MEM
--------------
  0  84%  66%
  1  96%  69%
  2  98%  69%
  3   0%  35%
  4   0%  35%
  5   0%   0%
  6   0%   1%
  7   0%   1%
available [5]


In [25]:
tf.logging.set_verbosity(tf.logging.DEBUG)
# load the dataset
dataset = cifar100.get_split('train', FLAGS.data_dir)

# load batch of dataset
batch_queue = load_batch(
    dataset,
    FLAGS.batch_size,
    is_training=True)
images, labels = batch_queue.dequeue()
slim.summary.image('input/image', images)

# tf.get_variable_scope().reuse_variables()
# run the image through the model
predictions, end_points = resnet50(images, classes=dataset.num_classes)
tf.get_variable_scope().reuse_variables()

    
# get the cross-entropy loss
one_hot_labels = slim.one_hot_encoding(
    labels,
    dataset.num_classes)

loss_100 = tf.losses.softmax_cross_entropy(
    logits=predictions,
    onehot_labels=one_hot_labels)

labels_coarse = tf.py_func(f2c, [labels], tf.int64)
# labels_coarse= tf.reshape(tf.concat(tf.constant(1,tf.int64), labels_coarse), [ 1,2])
labels_coarse = tf.reshape(labels_coarse, labels.shape)
labels_fine = tf.py_func(c2f, [labels_coarse], tf.int64)
labels_fine = tf.reshape(labels_fine, labels.shape.as_list() + [5, ])

one_hot_labels_coarse = tf.reduce_sum(
        tf.reshape(
            tf.to_int64(
                slim.one_hot_encoding(
                    tf.reshape(labels_fine, (-1,)),
                    num_classes=dataset.num_classes
                )
            ),
            labels.shape.as_list() + [5, -1]),
        axis=1
)
loss_20 = tf.losses.log_loss(
        predictions=tf.nn.softmax(predictions),
        labels=one_hot_labels_coarse,
        weights=FLAGS.beta,
        loss_collection=None if not FLAGS.multi_loss else tf.GraphKeys.LOSSES
    )

bs = labels_fine.shape[0]
predictions_l = []
for ind in range(5):
    sel = tf.stack([tf.range(bs, dtype=tf.int64), labels_fine[:, ind]], axis=1)
    predictions_l.append(tf.gather_nd(predictions, sel))
predictions_group = tf.stack(predictions_l, axis=1)

labels_group_one_hot = tf.equal(labels_fine, tf.expand_dims(labels, axis=-1))
labels_group_one_hot = tf.to_int64(labels_group_one_hot)

loss_group = tf.losses.softmax_cross_entropy(
    logits=predictions_group,
    onehot_labels=labels_group_one_hot,
    weights=FLAGS.gamma,
    loss_collection=None if not FLAGS.multi_loss else tf.GraphKeys.LOSSES
)

<tf.Tensor 'input/image:0' shape=() dtype=string>

In [26]:
sess=tf.Session(config=utils.allow_growth_config())
_=tf.train.queue_runner.start_queue_runners(sess)

In [27]:
sess.run(tf.global_variables_initializer())
variables_to_restore = slim.get_variables_to_restore(
        exclude=[".*logits.*", ".*Ftrl.*", '.*Momentum.*', 'global_step'])
FLAGS.checkpoint_path='/home/wangxinglu/prj/few-shot/output/multiloss-dbg/model.ckpt-85745'
init_assign_op, init_feed_dict = slim.assign_from_checkpoint(
        FLAGS.checkpoint_path, variables_to_restore)

print 'init from pretrained model'
sess.run([init_assign_op, ], init_feed_dict)

init from pretrained model


[None]

In [29]:
pred=_28[2]
ohlc=_28[4]

In [40]:
pred.shape,ohlc.shape

def softmax(x):
    # return np.exp(x) / np.sum(np.exp(x), axis=1).reshape(-1, 1)
    x = np.array(x)
    ex = np.exp(x - x.max(axis=1).reshape(-1, 1))
    # print ex.shape
    return ex / ex.sum(axis=1).reshape(-1, 1)
predt=softmax(pred)


((1, 100), (1, 100))

In [45]:
tt=predt.squeeze()[ohlc.squeeze().astype(bool)]
-np.log(tt.sum())

2.992187